# Using PyBIDS to Summarize a BIDS Dataset
_Project: 22Q_812481_ <br>
_Author: Katja Zoner_ <br>
_Date: 03/15/2021_  <br>

In [ ]:
import os
import pandas as pd
from bids import BIDSLayout, BIDSValidator
from bids.reports import BIDSReport

project = '/Users/kzoner/BBL/projects/22Q_812481'
bids_root = os.path.join(project,'data','bids_directory')

# Generate BIDSLayout obj from bids root directory
layout = BIDSLayout(bids_root)


In [134]:
# Get basic info from layout object
layout

BIDS Layout: ...22Q_812481/data/bids_directory | Subjects: 130 | Sessions: 131 | Runs: 225

## Project 22Q contains 130 Subjects and 131 Sessions total.

__*Note:*__ _`sub-16095` is the only subject in 22Q with longitudinal data! Their first session was in 2011, and their second session was in 2014._

In [ ]:
# Convert layout object to pandas df
df = layout.to_df()

# Edit df so that ASL files are of datatype 'perf' --> apparently pybids hasn't been updated for ASL data
is_perf = (df['acquisition']=='se')
df.loc[is_perf,'datatype']='perf'

df.head()

In [ ]:
# Merge run/task info into suffix col for pivot table
is_rest = (df['task']=='rest')
df.loc[is_rest,'suffix']='rest_bold'

is_idemo = (df['task']=='idemo')
df.loc[is_idemo,'suffix']='idemo_bold'

is_run1 = (df['run']==1)
df.loc[is_run1,'suffix']='run-01'

is_run2 = (df['run']==2)
df.loc[is_run2,'suffix']='run-02'

# Rename cols for pivot table
df.columns = ['File Count', 'acquisition', 'modality', 'extension', 'fmap', 'run', 'session', 'subject', 'scan', 'task']

## The following code creates and displays a summary table of the `.nii.gz` files in 22Q. <br>
For each subject, display file count by modality and scan type. <br>
See bottom row for project totals.


In [153]:
summary_table = pd.pivot_table(
    df[df['extension']=='nii.gz'], 
    index = ['subject'], 
    columns = ['modality','scan'], 
    values = ['File Count'], 
    aggfunc = pd.Series.nunique, 
    fill_value = 0, 
    margins = True, 
    margins_name = 'Total'
    )

summary_table['File Count']

modality anat    dwi              fmap                                func  \
scan      T1w run-01 run-02 magnitude1 magnitude2 phase1 phase2 idemo_bold   
subject                                                                      
109741      1      1      1          0          0      0      0          0   
11706       1      1      1          0          0      0      0          0   
11936       1      1      1          0          0      0      0          0   
12410       1      1      1          0          0      0      0          0   
12835       1      1      1          0          0      0      0          0   
...       ...    ...    ...        ...        ...    ...    ...        ...   
18379       1      1      1          1          1      1      0          1   
19434       1      1      1          1          1      1      0          1   
19475       1      1      1          1          1      1      0          1   
19489       1      0      0          1          1      1      0          0   
Total     131    115    112         41         41     43     14         45   

modality           perf Total  
scan     rest_bold  asl        
subject                        
109741           1    1     5  
11706            1    1     5  
11936            1    1     5  
12410            1    1     5  
12835            1    1     5  
...            ...  ...   ...  
18379            1    1     9  
19434            1    1     9  
19475            1    1     9  
19489            0    1     5  
Total          117  127   786  

[131 rows x 11 columns]